In [37]:
# !git clone https://github.com/keras-team/keras-tuner


In [4]:
cd keras-tuner

C:\Users\srika\Desktop\ML\PROJECT\Mask Recognition\keras-tuner


In [5]:
!pip install .

Processing c:\users\srika\desktop\ml\project\mask recognition\keras-tuner
  Created wheel for keras-tuner: filename=keras_tuner-1.0.3-py3-none-any.whl size=94520 sha256=1a12b5936d4ee872299152791c1e4339db0940dfb1b57b9a4057081785fe9051
  Stored in directory: c:\users\srika\appdata\local\pip\cache\wheels\24\39\2d\24b5a62ec12d615affac337986791b34b0845eccfda8eeea1d
Successfully built keras-tuner
  Attempting uninstall: keras-tuner
    Found existing installation: keras-tuner 1.0.3
    Uninstalling keras-tuner-1.0.3:
      Successfully uninstalled keras-tuner-1.0.3


In [6]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout

In [7]:
from kerastuner import HyperModel


class CNNHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        model = keras.Sequential()
        model.add(
                Conv2D(
        filters=hp.Int('conv_1_filter', min_value=8, max_value=32, step=4), 
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(150,150,1)
                     )
                 )
        model.add(MaxPooling2D(pool_size=2))
        model.add(
                Conv2D(
        filters=hp.Int('conv_2_filter', min_value=4, max_value=16, step=2),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
                      )
                 )
        model.add(MaxPooling2D(pool_size=2))
        model.add(Flatten())
        model.add(
                Dropout(
                rate=hp.Float(
                    'dropout_3',
                    min_value=0.0,
                    max_value=0.5,
                    default=0.25,
                    step=0.05
                            )
                      )
                )
        """
        model.add(
                  Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
                       )
                 )
        """
        model.add(Dense(2,activation='softmax'))

        model.compile(
            optimizer=keras.optimizers.Adam(
                hp.Float(
                    'learning_rate',
                    min_value=1e-4,
                    max_value=1e-2,
                    sampling='LOG',
                    default=1e-3
                )
            ),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        return model




In [8]:
from kerastuner.tuners import RandomSearch

NUM_CLASSES = 2
INPUT_SHAPE = (150,150,1)
MAX_TRIALS=5
N_EPOCH_SEARCH = 3

hypermodel = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)

tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    max_trials=MAX_TRIALS,
    directory='random_search_output',
    project_name='mask_recog_hyper_optimized'
)

INFO:tensorflow:Reloading Oracle from existing project random_search_output\mask_recog_hyper_optimized\oracle.json
INFO:tensorflow:Reloading Tuner from random_search_output\mask_recog_hyper_optimized\tuner0.json


In [9]:
from imutils import paths
import os
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
imagePaths = list(paths.list_images(r'C:\Users\srika\Desktop\ML\PROJECT\Mask Recognition\dataset'))
data = []
labels = []
# loop over the image paths
for imagePath in imagePaths:
    # extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2]

    # load the input image (150x150) and preprocess it (arbitrary in our case)
    image = load_img(imagePath, color_mode='grayscale',target_size=(150,150))

    image = img_to_array(image) #1,150,150

    # update the data and labels lists, respectively
    data.append(image)
    labels.append(label)
    
# convert the data and labels to NumPy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)

c:\users\srika\appdata\local\programs\python\python38\lib\site-packages\PIL\Image.py:962: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [30]:
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.20, stratify=labels, random_state=42)

In [31]:
tuner.search(trainX,trainY,batch_size=4 ,epochs=N_EPOCH_SEARCH, validation_split=0.1)

Trial 5 Complete [00h 01m 03s]
val_accuracy: 0.5570032596588135

Best val_accuracy So Far: 0.9348534345626831
Total elapsed time: 00h 05m 33s
INFO:tensorflow:Oracle triggered exit


In [32]:
model=tuner.get_best_models(num_models=1)[0]

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 24)      240       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 24)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 14)        3038      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 14)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18144)             0         
_________________________________________________________________
dropout (Dropout)            (None, 18144)             0         
_________________________________________________________________
dense (Dense)                (None, 2)                 3

In [34]:
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

INIT_LR = 1e-4
EPOCHS = 20
BS = 32

H = model.fit(
    aug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=EPOCHS)

print('Done')

Epoch 1/20
95/95 [==============================] - 26s 243ms/step - loss: 0.5556 - accuracy: 0.7703 - val_loss: 0.2285 - val_accuracy: 0.9113
Epoch 2/20
95/95 [==============================] - 14s 145ms/step - loss: 0.4425 - accuracy: 0.7954 - val_loss: 0.2240 - val_accuracy: 0.9100
Epoch 3/20
95/95 [==============================] - 20s 214ms/step - loss: 0.4272 - accuracy: 0.8041 - val_loss: 0.2189 - val_accuracy: 0.9179
Epoch 4/20
95/95 [==============================] - 15s 159ms/step - loss: 0.3898 - accuracy: 0.8223 - val_loss: 0.2166 - val_accuracy: 0.9166
Epoch 5/20
95/95 [==============================] - 23s 240ms/step - loss: 0.3634 - accuracy: 0.8341 - val_loss: 0.2026 - val_accuracy: 0.9192
Epoch 6/20
95/95 [==============================] - 24s 249ms/step - loss: 0.3532 - accuracy: 0.8396 - val_loss: 0.1881 - val_accuracy: 0.9231
Epoch 7/20
95/95 [==============================] - 20s 212ms/step - loss: 0.3269 - accuracy: 0.8678 - val_loss: 0.1816 - val_accuracy: 0.9283

In [35]:
%cd "C:\Users\srika\Desktop\ML\PROJECT\Mask Recognition"

C:\Users\srika\Desktop\ML\PROJECT\Mask Recognition


In [36]:
model.save('mask_recog_hyperparameter_tuned.h5')